In [1]:
pip install -r ../../requirements.txt


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [3]:
from zkstats.core import create_dummy, verifier_define_calculation, prover_gen_settings, setup, prover_gen_proof, verifier_verify, get_data_commitment_maps

In [4]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
sel_data_path = os.path.join('prover/sel_data.json')
# this is just dummy random value
sel_dummy_data_path = os.path.join('shared/sel_dummy_data.json')

=======================  ZK-STATS FLOW =======================

In [5]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')
create_dummy(data_path, dummy_data_path)


In [22]:
scales = [5]
selected_columns = ['x1', 'y']
commitment_maps = get_data_commitment_maps(data_path, scales)

In [23]:
# Verifier/ data consumer side: send desired calculation
from zkstats.computation import computation_to_model, State


def computation(s: State, data: list[torch.Tensor]) -> torch.Tensor:
    x, y = data[0], data[1]
    return s.linear_regression(x, y)

error = 0.02
_, verifier_model = computation_to_model(computation, error)

verifier_define_calculation(dummy_data_path, selected_columns, sel_dummy_data_path, verifier_model, verifier_model_path)

In [24]:
# Prover/ data owner side
_, prover_model = computation_to_model(computation, error)

prover_gen_settings(data_path, selected_columns, sel_data_path, prover_model,prover_model_path, scales, "resources", settings_path)

Using 6 columns for non-linearity table.
Using 6 columns for non-linearity table.
Using 6 columns for non-linearity table.
Using 12 columns for non-linearity table.
Using 12 columns for non-linearity table.
Using 12 columns for non-linearity table.


 <------------- Numerical Fidelity Report (input_scale: 5, param_scale: 5, scale_input_multiplier: 1) ------------->

+-------------+--------------+-------------+-----------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| mean_error  | median_error | max_error   | min_error | mean_abs_error | median_abs_error | max_abs_error | min_abs_error | mean_squared_error | mean_percent_error | mean_abs_percent_error |
+-------------+--------------+-------------+-----------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| 0.006681005 | 0.0076099634 | 0.012433052 | 0         

==== Generate & Calibrate Setting ====
scale:  [5]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":5,"param_scale":5,"scale_rebase_multiplier":1,"lookup_range":[-11984,368458],"logrows":19,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private","div_rebasing":false,"rebase_frac_zero_constants":false,"check_mode":"UNSAFE"},"num_rows":7872,"total_assignments":468,"total_const_size":17,"model_instance_shapes":[[1],[1,2,1]],"model_output_scales":[0,5],"model_input_scales":[5,5],"module_sizes":{"kzg":[],"poseidon":[7872,[2]]},"required_lookups":[{"GreaterThan":{"a":0.0}},"Abs"],"required_range_checks":[[-16,16]],"check_mode":"UNSAFE","version":"9.1.0","num_blinding_factors":null,"timestamp":1709452000287}


In [25]:
# Here verifier & prover can concurrently call setup since all params are public to get pk.
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
prover_gen_proof(prover_model_path, sel_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

==== setting up ezkl ====
Time setup: 57.17967200279236 seconds
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 1.5625
witness result 2 : 9.53125
==== Generating Proof ====
proof:  {'instances': [['cdb07195b0e3b568c9d622dde480fb97d8a12b0e4ceb8fbbf155e0cb4c58720a', '81c924e0484c8b03dc1fb951bf90b87c4947f9f6ef4918bcacc3fc687376cf06', '0100000000000000000000000000000000000000000000000000000000000000', '3200000000000000000000000000000000000000000000000000000000000000', '3101000000000000000000000000000000000000000000000000000000000000']], 'proof': '0x2468f03b2151371062ce905abda32cd48cfbb06b24f3e73261b211e40d6322d6224f8365fb1c70ec51e2ef8f8fa97e61ac4b3b8a53d1e5f3b07b7fb3b8f0c5b62da4c48346d2b4ab5f67a872e0306dd022995860253e91c913e693f30cf77ff314bb7d619bf8159212afa5a27cdc369369b6fedf854ea5e95f3a4921e14707952f026bfd134488ec24344e5c48b3f93f6c24c2982ef1500724dbf10f86de29b905466ae3c9512075548484ecbc45ce38cc909fdb3263cf5e912a60f55424cb5e28cf2cf2a8375cb803bda7e0634608c7ccc15127e4a

In [26]:
# Verifier verifies
res = verifier_verify(proof_path, settings_path, vk_path, selected_columns, commitment_maps)
print("Verifier gets result:", res)

Verifier gets result: [1.5625, 9.53125]
